# Generate Data

Generates UMAP coordinates and model from HuggingFace datasets or a JSON file.

In this notebook, we use the [Wikipedia sentences](https://huggingface.co/datasets/sentence-transformers/wikipedia-en-sentences) (HuggingFace) + [CHI 2024 paper titles](https://observablehq.com/@john-guerra/chi2024-papers) (JSON) datasets as examples. You may need to adjust the code to fit your needs.

## Helpers and imports

In [1]:
from datasets import load_dataset
import pandas as pd
import os
import json
import pickle
import numpy as np

import torch
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizer, PreTrainedModel
import vec2text

import umap
import plotly.express as px
from sklearn.cluster import KMeans

from openai import OpenAI
import ast

c:\Users\euije\.virtualenvs\ml-interactive-data-augmentation-oGByDv0u\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# check if models folder exists
if not os.path.exists("../models"):
    os.makedirs("../models")

In [3]:
# load the api keys from the secrets file
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    print("API keys loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API keys loaded.


In [4]:
OPENAI_MODEL = "gpt-4o-mini"
llm = OpenAI(api_key=open_ai_key)

In [5]:
# Setting the random seed for reproducibility
random_state = 42
sample_size = 1000
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [13]:
# check for cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### generate embeddings

In [7]:
def get_gtr_embeddings(text_list,
                       encoder: PreTrainedModel,
                       tokenizer: PreTrainedTokenizer) -> torch.Tensor:

    inputs = tokenizer(text_list,
                       return_tensors="pt",
                       max_length=128,
                       truncation=True,
                       padding="max_length",).to(device)

    with torch.no_grad():
        model_output = encoder(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        hidden_state = model_output.last_hidden_state
        embeddings = vec2text.models.model_utils.mean_pool(hidden_state, inputs['attention_mask'])

    return embeddings

encoder = AutoModel.from_pretrained("sentence-transformers/gtr-t5-base").encoder.to(device)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/gtr-t5-base")

In [8]:
def generate_embeddings(sentences, dataset_name):
    # batch embedding process
    batch_size = 100
    embeddings = []
    for i in range(0, len(sentences), batch_size):
        embeddings.append(get_gtr_embeddings(sentences[i:i+batch_size], encoder, tokenizer))

    # combine embeddings into one tensor
    embeddings = torch.cat(embeddings, dim=0)

    # normalize embeddings
    norms = torch.norm(embeddings, dim=1)
    embeddings /= norms[:, None]

    # save embeddings to file 
    torch.save(embeddings, f"{dataset_name}/{dataset_name}_embeddings.pt")

    print(f"Embeddings for {dataset_name} generated and saved to file.")
    return embeddings

### UMAP

In [9]:
def run_umap(embeddings, sentences, dataset_name, n_neighbors=100, min_dist=0.1):
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2, metric='cosine', random_state=random_state)
    reducer = umap_model.fit(embeddings.cpu())
    umap_embeddings = reducer.transform(embeddings.cpu())

    # save results to df
    df_res = pd.DataFrame()
    df_res['sentence'] = sentences
    df_res['umap_x'] = umap_embeddings[:,0]
    df_res['umap_y'] = umap_embeddings[:,1]

    # save umap model
    umap_path = f'../models/{dataset_name}_umap_reducer'
    umap_pickle = open(umap_path, 'wb')
    pickle.dump(reducer, umap_pickle)
    umap_pickle.close()

    print(f"UMAP model for {dataset_name} saved to file.")
    return df_res

### Cluster

In [10]:
def label_clusters(cluster_samples):
    cluster_examples = "" # input to LLM
    for cluster, samples in cluster_samples.items():
        cluster_examples += f'Cluster {cluster}\n'
        cluster_examples += '----------------\n'
        for sample in samples:
            cluster_examples += sample + '\n'
        cluster_examples += '\n'

    system_prompt = f"""Given the example 10 sentences in each cluster, please assign a short phrase to describe each cluster. 
                    Format your answer as a dict where the key is the cluster number and the value is the phrase.
                    Just output the dict, nothing else."""
    user_prompt = f"{cluster_examples}"
    temperature = 0.1

    # get completion from LLM (ask it to label the clusters)
    completion = llm.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    res = completion.choices[0].message.content

    # read res as a dictionary
    label_map = ast.literal_eval(res)
    print("Cluster labels generated.")
    return label_map


In [11]:
def get_clusters(df_res, embeddings, dataset_name, num_clusters=10):
    # create copy of df_res
    df_cluster = df_res.copy()

    # create a kmeans model
    kmeans = KMeans(n_clusters=num_clusters, random_state=random_state)

    # fit the model
    kmeans.fit(embeddings)

    # get the cluster labels
    cluster_labels = kmeans.labels_
    df_cluster['cluster'] = cluster_labels

    # randomly sample 10 sentences from each cluster
    # and add them to a dictionary where the key is the cluster number
    cluster_samples = {}
    for cluster in range(num_clusters):
        cluster_samples[cluster] = df_cluster[df_cluster['cluster'] == cluster].sample(10)['sentence'].tolist()

    # get the labels for the clusters
    label_map = label_clusters(cluster_samples)

    # map df_cluster['cluster'] to the labels
    df_cluster['cluster'] = df_cluster['cluster'].map(label_map)

    # save data to json
    df_cluster.to_json(f'{dataset_name}/{dataset_name}_data.json', orient='records')
    
    print(f"Cluster info and umap coords for {dataset_name} saved to file.")
    return df_cluster

In [12]:
def visualize_clusters(df_cluster):
    # create interactive visualization of the embeddings
    fig = px.scatter(df_cluster, x='umap_x', y='umap_y', hover_data=['sentence'], color='cluster')

    width = 1000

    # add a title and change the layout
    fig.update_layout(title='UMAP of sentence embeddings with clusters',
        autosize=False,
        width=width,
        height=400,
    )

    # show the plot
    fig.show()

## Wikipedia Sentences (HuggingFace Example)

In [ ]:
dataset_to_load = "sentence-transformers/wikipedia-en-sentences" # TODO: REPLACE WITH ANY HUGGINGFACE DATASET
data = load_dataset(dataset_to_load)

dataset_name = "wiki" # TODO: REPLACE WITH YOUR DATASET NAME

c:\Users\euije\.virtualenvs\ml-interactive-data-augmentation-oGByDv0u\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\euije\.cache\huggingface\hub\datasets--sentence-transformers--wikipedia-en-sentences. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xe

In [15]:
# check if folder exists
if not os.path.exists(dataset_name):
    os.makedirs(dataset_name)

In [16]:
# sample n rows from the dataset
n = 10000
sample = data['train'].shuffle(seed=42).select(range(n))

In [17]:
df = pd.DataFrame(sample[:sample_size])
# rename the column to 'Sentence'
df = df.rename(columns={'sentence': 'Sentence'})
df.head()

,Sentence
0,"The EP, and ""Kaleidoscope World"" in particular..."
1,Reiner used an ink wash to give the strip a gr...
2,"He has competed in IMSA GT, sprint car racing,..."
3,She is a co-founder of Sweatshop Watch.
4,The traditional notion of the application of t...


In [18]:
sentences = df['Sentence'].to_list() # get sentences from df

In [19]:
# generate embeddings
embeddings = generate_embeddings(sentences, dataset_name)

Embeddings for wiki generated and saved to file.


In [20]:
# get umap coords
df_res = run_umap(embeddings, sentences, dataset_name)

c:\Users\euije\.virtualenvs\ml-interactive-data-augmentation-oGByDv0u\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP model for wiki saved to file.


In [21]:
# cluster the data
df_cluster = get_clusters(df_res, embeddings, dataset_name)

Cluster labels generated.
Cluster info and umap coords for wiki saved to file.


In [22]:
# visualize the clusters
visualize_clusters(df_cluster)

## CHI 2024 Papers (JSON Example)

In [24]:
# load json data
file_name = "chi2024data.json" # TODO: REPLACE WITH YOUR FILE NAME
dataset_name = "chi" # TODO: REPLACE WITH YOUR DATASET NAME

with open(file_name, "r", encoding="utf-8") as f:
    data = json.load(f)

In [25]:
# check if folder exists
if not os.path.exists(dataset_name):
    os.makedirs(dataset_name)

In [26]:
papers = data["contents"]

# get title from each paper
titles = [paper["title"].strip() for paper in papers]

len(titles)

1763

In [27]:
# randomly sample 1000 transcripts from this dataset
np.random.seed(random_state)
sampled_indices = np.random.choice(len(titles), sample_size, replace=False)

sentences = titles

In [28]:
sentences[:5]

['Seam Work and Simulacra of Societal Impact in Networking Research: A Critical Technical Practice Approach',
 'Make This! Introduction to Electronics Prototyping Using Arduino',
 '“I was able to give her the confidence”: Reciprocal Capacity Building in a Community-based Program for Digital Engagement',
 'ClassInSight: Designing Conversation Support Tools to Visualize Classroom Discussion for Personalized Teacher Professional Development',
 'Playing on Hard Mode: Accessibility, Difficulty and Joy in Video Game Adoption for Gamers with Disabilities']

In [29]:
# generate embeddings
embeddings = generate_embeddings(sentences, dataset_name)

Embeddings for chi generated and saved to file.


In [30]:
# get umap coords
df_res = run_umap(embeddings, sentences, dataset_name)

c:\Users\euije\.virtualenvs\ml-interactive-data-augmentation-oGByDv0u\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



UMAP model for chi saved to file.


In [31]:
# cluster the data
df_cluster = get_clusters(df_res, embeddings, dataset_name, num_clusters=25)

Cluster labels generated.
Cluster info and umap coords for chi saved to file.


In [32]:
# visualize the clusters
visualize_clusters(df_cluster)

# HCSLAB (json)

In [14]:
# load json data
file_name = "hcslab.json" # TODO: REPLACE WITH YOUR FILE NAME
dataset_name = "hcslab" # TODO: REPLACE WITH YOUR DATASET NAME

with open(file_name, "r", encoding="utf-8") as f:
    data = json.load(f)

In [15]:
# check if folder exists
if not os.path.exists(dataset_name):
    os.makedirs(dataset_name)

In [16]:
papers = data["contents"]

# get title from each paper
titles = [paper["title"].strip() for paper in papers]

len(titles)

30

In [ ]:
# randomly sample 1000 transcripts from this dataset
np.random.seed(random_state)
sampled_indices = np.random.choice(len(titles), len(titles), replace=False)

sentences = titles

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
sentences[:5]

In [ ]:
# generate embeddings
embeddings = generate_embeddings(sentences, dataset_name)

In [ ]:
# get umap coords
df_res = run_umap(embeddings, sentences, dataset_name)

In [ ]:
# cluster the data
df_cluster = get_clusters(df_res, embeddings, dataset_name, num_clusters=25)

In [ ]:
# visualize the clusters
visualize_clusters(df_cluster)